In [1]:
from string import punctuation
from razdel import sentenize
from razdel import tokenize as razdel_tokenize
import numpy as np
from collections import Counter
from nltk.tokenize import sent_tokenize

In [2]:
dvach = open('2ch_corpus.txt', encoding='utf-8').read()

In [3]:
# Функция нормализации

def normalize(text):
    normalized_text = [word.text.strip(punctuation) for word \
                                                            in razdel_tokenize(text)]
    normalized_text = [word.lower() for word in normalized_text if word and len(word) < 20 ]
    return normalized_text

In [4]:
norm_dvach = normalize(dvach)

In [5]:
# Превращение абсолютных частот в вероятности

vocab_dvach = Counter(norm_dvach)
probas_dvach = Counter({word:c/len(norm_dvach) for word, c in vocab_dvach.items()})

In [6]:
def ngrammer(tokens, n=3):
    ngrams = []
    for i in range(0,len(tokens)-n+1):
        ngrams.append(' '.join(tokens[i:i+n]))
    return ngrams

In [8]:
sentences_dvach = [['<start>'] + ['<start>'] + normalize(text) + ['<end>'] for text in sent_tokenize(dvach)]
sentences_dvach[:3]

[['<start>',
  '<start>',
  'анимублядский',
  'webm-треддля',
  'приличных',
  'анимублядей',
  'и',
  'прочих',
  'аутистов',
  '<end>'],
 ['<start>',
  '<start>',
  'безграмотное',
  'быдло',
  'с',
  'дубляжом',
  'войсовером',
  'порнографией',
  'и',
  'котиками',
  'советы',
  'мерзких',
  'мокрописечников',
  'вниманиебляди',
  'всех',
  'видов',
  'и',
  'прочее',
  'непотребство',
  'отправляется',
  'в',
  'порнотред',
  'ссылка',
  'для',
  'поиска',
  'с',
  'o',
  'уса',
  'видео',
  'сохраняем',
  'кадр',
  'правый',
  'клик',
  'по',
  'видео',
  'и',
  'ищем',
  'его',
  'на',
  'для',
  'воспроизведения',
  'webm',
  'с',
  '10-битным',
  'цветом',
  'нужно',
  'установить',
  'плагин',
  'vlc',
  'и',
  'отключить',
  'встроенный',
  'в',
  'браузер',
  'плеер',
  'media',
  '<end>'],
 ['<start>', '<start>', 'webm', '<end>']]

In [9]:
trigrams_dvach = Counter()
bigrams_dvach = Counter()
unigrams_dvach = Counter()


for sentence in sentences_dvach:
    trigrams_dvach.update(ngrammer(sentence,3))
    bigrams_dvach.update(ngrammer(sentence,2))
    unigrams_dvach.update(sentence)

In [10]:
unigrams_dvach = Counter()
bigrams_dvach = Counter()
trigrams_dvach = Counter()

for sentence in sentences_dvach:
    trigrams_dvach.update(ngrammer(sentence))
    bigrams_dvach.update(ngrammer(sentence,2))
    unigrams_dvach.update(sentence)

In [11]:
from scipy.sparse import lil_matrix, csr_matrix

In [12]:
matrix_dvach = lil_matrix((len(bigrams_dvach), len(unigrams_dvach))) 
            
id2word_dvach = list(unigrams_dvach)
word2id_dvach = {word:i for i, word in enumerate(id2word_dvach)}

id2bigram_dvach = list(bigrams_dvach)
bigram2id_dvach = {word:i for i, word in enumerate(id2bigram_dvach)}

for ngram in trigrams_dvach:
    word1, word2, word3 = ngram.split()
    bigram = word1 + ' ' + word2

    matrix_dvach[bigram2id_dvach[bigram],word2id_dvach[word3]] =  (trigrams_dvach[ngram]/bigrams_dvach[bigram])

In [13]:
matrix_dvach = csr_matrix(matrix_dvach)

In [14]:
np.random.choice(list(range(matrix_dvach.shape[1])), 
                 p=matrix_dvach[1].toarray()[0])

2

In [15]:
def generate(matrix, id2bigram, id2word, bigram2id, n=100, start='<start> <start>'):
    text = []
    current_idx = bigram2id[start]
    for i in range(n):
        p = matrix[current_idx].toarray()[0]
        chosen = np.random.choice(list(range(matrix.shape[1])), p = p)
        text.append(id2word[chosen])
        if id2word[chosen] == '<end>':
            current_idx = bigram2id[start]
        else:
            part = id2bigram[current_idx] + ' ' + id2word[chosen]
            part = ' '.join(part.split()[1:])
            current_idx = bigram2id[part]
    
    return ' '.join(text)


In [16]:
print(generate(matrix_dvach, id2bigram_dvach, id2word_dvach, bigram2id_dvach, start='<start> <start>').replace('<end>', '\n'))

малацца тонко 
 хомячье в край ебанулось 
 спасибо схороню этот тред также можно с 1 чуваком без всякого вреда для себя открыл мне её произвести а он в моем случае оба варианта проигрыш на самом деле дышит и сильнее всего тревогу лечат никотин и алкоголь добьют уже лет 20 блять этот чувак типа не они одни занялись тогда ебанатством 
 скажите где это 
 а вообще жри сажу зелёный лампавая он няша с сум хочу игру в общем дискас самостоятельная сборка попахивает чем-то типа ядерной бомбы или как это принято в тех местах внезапно новые кнопки какие-то и все


In [17]:
print(generate(matrix_dvach, id2bigram_dvach, id2word_dvach, bigram2id_dvach, start='<start> <start>').replace('<end>', '\n'))

не будет играть 
 листвачек скромностьчек бунтарьчек наркотачек мордобоичек вобрал в себя и что блядь сувать в пизду со своей формальной логикой потом и сообщника 
 ну я думаю хозяин это машины явно не все 
 манька поддавашка 
 должно быть даже что утерянная давно технология не очень большой 
 кто это вообще на са наверное даже не в курсе подробностей и таки устроиться там работать 
 давайте пишите всякую дичь а я и сказал веди меня по крайней мере весь хайп улегся 
 хотим слушать как сосед пердит по ночам деревенское быдло забавы ради та еще мечта битарда 



In [18]:
print(generate(matrix_dvach, id2bigram_dvach, id2word_dvach, bigram2id_dvach, start='<start> <start>').replace('<end>', '\n'))

когда вернулся домой за поребрик 
 17 
 парни выручайте никак не мог быть установлен какой-то порог критческой боимассы 
 зачем быть во френдзоне но такой хуйней не нужно его трогать 
 расскажите нам ещё что-нибудь высри бомбящий петух 
 наверное сон был очень расстроен и сказал что художником никогда не бомбило так при этом ничего плохого в убийстве животных 
 лазером удалось вывести из страны или на той мумия 
 сразу скажу нормальные меня же тогда разместили вертолёты там 
 хуйню ты слушаешь разновидность миталкора 
 он может учиться в хорошей форме я не удивлен 
 играю немного опыта


In [19]:
def perplexity(probas):
    p = np.exp(np.sum(probas))
    N = len(probas)
    
    return p**(-1/N) 

In [22]:
all_perplexity = []
for sent in sentences_dvach:
    probs = []
    for ngram in ngrammer(sent, 3):
        word1, word2, word3 = ngram.split()
        bigram = word1 + ' '+word2
        
        if ngram in trigrams_dvach and bigram in bigrams_dvach:
            probs.append(np.log(trigrams_dvach[ngram]/bigrams_dvach[bigram]))
        else:
            probs.append(np.log(0.00001))
    if perplexity(probs)!= np.inf: 
        all_perplexity.append(perplexity(probs))

In [23]:
np.mean(all_perplexity)

22.133904387208297